In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ats-score-dataset/dataset.csv


In [2]:
import pandas as pd
df = pd.read_csv('/kaggle/input/ats-score-dataset/dataset.csv')

In [5]:
df.head()

,ID,Name,Role,Transcript,Resume,decision,Reason_for_decision,Job_Description
0,jasojo159,Jason Jones,E-commerce Specialist,"Interviewer: Good morning, Jason. It's great t...",Here's a professional resume for Jason Jones:\...,reject,Lacked leadership skills for a senior position.,Be part of a passionate team at the forefront ...
1,annma759,Ann Marshall,Game Developer,Interview Scene\n\nA conference room with a ta...,Here's a professional resume for Ann Marshall:...,select,Strong technical skills in AI and ML.,Help us build the next-generation products as ...
2,patrmc729,Patrick Mcclain,Human Resources Specialist,Interview Setting: A conference room in a medi...,Here's a professional resume for Patrick Mccla...,reject,Insufficient system design expertise for senio...,We need a Human Resources Specialist to enhanc...
3,patrgr422,Patricia Gray,E-commerce Specialist,Here's a simulated professional interview for ...,Here's a professional resume for Patricia Gray...,select,Impressive leadership and communication abilit...,Be part of a passionate team at the forefront ...
4,amangr696,Amanda Gross,E-commerce Specialist,Here's the simulated interview:\n\nInterviewer...,Here's a professional resume for Amanda Gross:...,reject,Lacked leadership skills for a senior position.,We are looking for an experienced E-commerce S...


In [6]:
# Step 1: Preprocess Dataset (Kaggle Version)
import re, pandas as pd, spacy
from pathlib import Path

# ─────────────────────────────
# Setup Paths
# ─────────────────────────────
RAW_CSV  = "/kaggle/input/ats-score-dataset/dataset.csv"
OUT_PATH = Path("/kaggle/working/preprocessed_dataset.csv")

# ─────────────────────────────
# Load spaCy model
# ─────────────────────────────
SPACY_MODEL = "en_core_web_sm"

try:
    nlp = spacy.load(SPACY_MODEL, disable=["ner", "parser"])
except OSError:
    from spacy.cli import download
    download(SPACY_MODEL)
    nlp = spacy.load(SPACY_MODEL, disable=["ner", "parser"])

STOP_WORDS = nlp.Defaults.stop_words
RE_NONWORD = re.compile(r"\W+")
RE_DIGITS = re.compile(r"\d+")

def clean(text):
    text = RE_NONWORD.sub(" ", text.lower())
    text = RE_DIGITS.sub(" ", text)
    doc = nlp(text)
    return " ".join(tok.lemma_ for tok in doc if tok.lemma_ not in STOP_WORDS and len(tok) > 1)

# ─────────────────────────────
# Load and Prepare Data
# ─────────────────────────────
df = pd.read_csv(RAW_CSV)

# Keep only relevant columns
cols_to_keep = ["ID", "Name", "Role", "Resume", "Job_Description", "decision", "Reason_for_decision"]
df = df[cols_to_keep].drop_duplicates(subset=["ID"])

# Clean text fields
print("🔄 Preprocessing Resume, JD, and Role ...")
df["Resume_Clean"] = df["Resume"].astype(str).map(clean)
df["JD_Clean"] = df["Job_Description"].astype(str).map(clean)
df["Role_Clean"] = df["Role"].astype(str).map(clean)

# ─────────────────────────────
# Save Preprocessed Dataset
# ─────────────────────────────
df.to_csv(OUT_PATH, index=False)
print(f"✅ Preprocessed dataset saved to → {OUT_PATH}")


🔄 Preprocessing Resume, JD, and Role ...
✅ Preprocessed dataset saved to → /kaggle/working/preprocessed_dataset.csv


In [7]:
import pandas as pd
df = pd.read_csv('/kaggle/working/preprocessed_dataset.csv')

In [8]:
df.head()

,ID,Name,Role,Resume,Job_Description,decision,Reason_for_decision,Resume_Clean,JD_Clean,Role_Clean
0,jasojo159,Jason Jones,E-commerce Specialist,Here's a professional resume for Jason Jones:\...,Be part of a passionate team at the forefront ...,reject,Lacked leadership skills for a senior position.,professional resume jason jones jason jones co...,passionate team forefront machine learning com...,commerce specialist
1,annma759,Ann Marshall,Game Developer,Here's a professional resume for Ann Marshall:...,Help us build the next-generation products as ...,select,Strong technical skills in AI and ML.,professional resume ann marshall ann marshall ...,help build generation product game developer w...,game developer
2,patrmc729,Patrick Mcclain,Human Resources Specialist,Here's a professional resume for Patrick Mccla...,We need a Human Resources Specialist to enhanc...,reject,Insufficient system design expertise for senio...,professional resume patrick mcclain patrick mc...,need human resource specialist enhance team te...,human resource specialist
3,patrgr422,Patricia Gray,E-commerce Specialist,Here's a professional resume for Patricia Gray...,Be part of a passionate team at the forefront ...,select,Impressive leadership and communication abilit...,professional resume patricia gray patricia gra...,passionate team forefront cloud computing comm...,commerce specialist
4,amangr696,Amanda Gross,E-commerce Specialist,Here's a professional resume for Amanda Gross:...,We are looking for an experienced E-commerce S...,reject,Lacked leadership skills for a senior position.,professional resume amanda gross amanda gross ...,look experienced commerce specialist join team...,commerce specialist
